In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score


import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [2]:
df = pd.read_csv(r"C:\Users\trush\OneDrive\Documents\WFU Grad School Info\BAN 6025 Machine Learning\Sky's the Limit\BAN6025Project2Data.csv")
df.head()

,Target_Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,...,0,1500,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,...,0,1500,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,...,0,200,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,...,0,200,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,...,0,200,0,INTERNET,3.743048,other,0,1,0,0


In [3]:
df = pd.get_dummies(df, columns=['X8', 'X18', 'X15', 'X25', 'X27'], drop_first=True, dtype=float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 48 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   Target_Y       1000000 non-null  int64  
 1   X1             1000000 non-null  float64
 2   X2             1000000 non-null  float64
 3   X3             1000000 non-null  int64  
 4   X4             1000000 non-null  int64  
 5   X5             1000000 non-null  int64  
 6   X6             1000000 non-null  float64
 7   X7             1000000 non-null  float64
 8   X9             1000000 non-null  int64  
 9   X10            1000000 non-null  float64
 10  X11            1000000 non-null  float64
 11  X12            1000000 non-null  float64
 12  X13            1000000 non-null  int64  
 13  X14            1000000 non-null  int64  
 14  X16            1000000 non-null  int64  
 15  X17            1000000 non-null  int64  
 16  X19            1000000 non-null  int64  
 17  X20      

In [9]:

def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out = 0.05,
                       verbose=True):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS

    Arguments:
        X - pandas.DataFrame of numeric features
        y - vector, series of the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions

    Returns: list of selected features

    Example Call: stepwise_selection(X, y)
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print(f'Add  {best_feature} with p-value {best_pval:.4f}')
        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Drop {worst_feature} with p-value {worst_pval:.4f}')
        if not changed:
           break
    return included

In [4]:
# Split features and target
X = df.drop(columns=['Target_Y'])
y = df['Target_Y']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)


In [5]:
chosen_columns = ['X5',
 'X28',
 'X23',
 'X27_windows',
 'X22',
 'X19',
 'X2',
 'X1',
 'X8_AC',
 'X18_BC',
 'X18_BB',
 'X18_BE',
 'X18_BD',
 'X29',
 'X17',
 'X3',
 'X16',
 'X15_CB',
 'X21',
 'X24',
 'X14',
 'X7',
 'X9',
 'X27_macintosh',
 'X15_CC',
 'X18_BF',
 'X27_other',
 'X31',
 'X25_TELEAPP',
 'X15_CD',
 'X15_CF',
 'X20',
 'X6',
 'X4',
 'X13',
 'X15_CE',
 'X18_BG',
 'X27_x11']
print(len(chosen_columns))

38


In [6]:
X_selected = X[chosen_columns]  

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.30, random_state=69, stratify=y)

# Apply SMOTE to balance the training set
smote = SMOTE(sampling_strategy='auto', random_state=69)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Convert resampled training data back to DataFrame
X_train_resampled = pd.DataFrame(X_train_resampled, columns=X_selected.columns)

In [7]:
model = Sequential()
model.add(Dense(19, activation='tanh', input_shape=(X_train_resampled.shape[1],)))  # Matches feature count
model.add(Dense(1, activation='sigmoid'))  # No need for input_shape

# Compile model
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using resampled training data
model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=64, verbose=1)


c:\Users\trush\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 15s 634us/step - accuracy: 0.8240 - loss: 0.3927
Epoch 2/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 17s 805us/step - accuracy: 0.8457 - loss: 0.3524
Epoch 3/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 17s 802us/step - accuracy: 0.8502 - loss: 0.3440
Epoch 4/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 18s 810us/step - accuracy: 0.8549 - loss: 0.3354
Epoch 5/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 18s 814us/step - accuracy: 0.8588 - loss: 0.3273
Epoch 6/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 18s 829us/step - accuracy: 0.8633 - loss: 0.3174
Epoch 7/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 18s 849us/step - accuracy: 0.8675 - loss: 0.3080
Epoch 8/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 20s 923us/step - accuracy: 0.8700 - loss: 0.3030
Epoch 9/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 20s 902us/step - accuracy: 0.8707 - loss: 0.3015
Epoch 10/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 19s 880us/step - accuracy: 0.8728 - loss: 0.2973
Epoch 11/100
21634/21634 ━━━━━━━━━━━━━━━━━━━━ 19s 879us/ste

In [24]:
# Predict on training and test sets
y_pred_probs_train = model.predict(X_train_resampled)
y_pred_train = (y_pred_probs_train > 0.475).astype(int)

y_pred_probs_test = model.predict(X_test)
y_pred_test = (y_pred_probs_test > 0.475).astype(int)

43268/43268 ━━━━━━━━━━━━━━━━━━━━ 24s 557us/step
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 6s 665us/step


In [25]:
# Print performance metrics
print('---Train Set---')
print(f'Accuracy: {accuracy_score(y_train_resampled, y_pred_train):.4f}')
print(f'Precision: {precision_score(y_train_resampled, y_pred_train):.4f}')
print(f'Recall: {recall_score(y_train_resampled, y_pred_train):.4f}')
print(f'F1 Score: {f1_score(y_train_resampled, y_pred_train):.4f}')
print(f'ROC AUC {roc_auc_score(y_train_resampled, y_pred_train)}')

print('---Test Set---')
print(f'Accuracy: {accuracy_score(y_test, y_pred_test):.4f}')
print(f'Precision: {precision_score(y_test, y_pred_test):.4f}')
print(f'Recall: {recall_score(y_test, y_pred_test):.4f}')
print(f'F1 Score: {f1_score(y_test, y_pred_test):.4f}')
print(f'ROC AUC {roc_auc_score(y_test, y_pred_test)}')

---Train Set---
Accuracy: 0.8840
Precision: 0.8760
Recall: 0.8947
F1 Score: 0.8852
ROC AUC 0.8840187496388744
---Test Set---
Accuracy: 0.8686
Precision: 0.0546
Recall: 0.6691
F1 Score: 0.1010
ROC AUC 0.7699480177203757
